# **Sentimental Analysis of Twitter tweets**
Project for Letsupgarde -Bale Veeresh Siva Sai

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json

In [2]:
import tweepy
import openpyxl
from tweepy import OAuthHandler

In [3]:
# credentials

consumer_key = "nucwiQk9GtsJfWFSGQjXFicnq"
consumer_secret = "psbIvsTINgr6eEu89jMe2FwsOM28Cs3KaVLWUfBjADkAfFfhZV"
access_token = "296102538-4rqAPJBZSl2dQW2Wd1qvciWfkNKFpv9GZ3IBp5t7"
access_token_secret = "FoCmyrWsWVeyxjksdRwqujVyzQGT9tZVjRGqJoav021MU"

**Authenticating with Twitter API**

In [4]:
# call API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

**Collecting Latest Tweets about Covid Vaccine using tweepy**

In [5]:
# Lets provide the query which I need to search

query = "Covid Vaccine India"

# Fetch tweets

Tweets = api.search(query, count=10, lang= 'en', exclude= 'retweets', tweet_mode= 'extended')

for tweet in Tweets:
    print(tweet)
    print("=================================================")

Status(_api=<tweepy.api.API object at 0x7fcab83068d0>, _json={'created_at': 'Sun Jul 04 17:40:47 +0000 2021', 'id': 1411741795411693568, 'id_str': '1411741795411693568', 'full_text': '@Prayagwale Why did the US refused to provide raw material for the COVID vaccine to India despite repeated requests from the Indian government?', 'truncated': False, 'display_text_range': [12, 143], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Prayagwale', 'name': 'Prayagraj', 'id': 851724981574782977, 'id_str': '851724981574782977', 'indices': [0, 11]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': 1411738112061894657, 'in_reply_to_status_id_str': '1411738112061894657', 'in_reply_to_user_id': 851724981574782977, 'in_reply_to_user_id_str': '851724981574782977', 'in_reply_to_screen_name': 'Prayagwale', 'user': {'id': 637

# Getting the Tweets and some of its related Attributes

In [6]:
df = pd.DataFrame(columns=[
    'Tweets', 'User', 'User_statuses_count',
    'user_followers', 'User_location', 'User_verified',
    'fav_count', 'rt_count', 'tweet_date'
])
df

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date


In [7]:
def stream(data, file_name):
    i=0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [9]:
stream(data=["Covid Vaccine India"], file_name='my_tweets')

In [10]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @HardeepSPuri: 🔬 Government further scales ...,Beinginformed.in,18915,293,My Country India,False,0,44,2021-07-04 17:46:31
1,RT @HardeepSPuri: 🔬 Government further scales ...,Mohsin Shaikh🇮🇳,13722,1805,"Pune, Maharashtra",False,0,44,2021-07-04 17:44:42
2,"RT @FaheemYounus: Great News:\n\nPfizer, Moder...",Ibn Anjar™,35074,874,"ÜT: -6.2130817,107.0288138",False,0,1539,2021-07-04 17:44:26
3,RT @HardeepSPuri: 🔬 Government further scales ...,VISHWANATH ACHARYA,26857,546,"Pune, Maharashtra, India",False,0,44,2021-07-04 17:44:19
4,RT @HardeepSPuri: 🔬 Government further scales ...,Astro-कुमकुम 🇮🇳,41656,1218,"New Delhi, India",False,0,44,2021-07-04 17:44:07


In [11]:
df.shape

(1000, 9)

In [12]:
from textblob import TextBlob

In [13]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [14]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity>0:
        return 'Positive'
    elif analysis.sentiment.polarity==0:
        return 'Neutral'
    else:
        return 'Negative'

In [15]:
df['clean_tweet'] = df['Tweets'].apply(lambda x: clean_tweet(x))

In [16]:
df[['Tweets','clean_tweet']].head()

,Tweets,clean_tweet
0,RT @HardeepSPuri: 🔬 Government further scales ...,RT Government further scales up India s Vaccin...
1,RT @HardeepSPuri: 🔬 Government further scales ...,RT Government further scales up India s Vaccin...
2,"RT @FaheemYounus: Great News:\n\nPfizer, Moder...",RT Great News Pfizer Moderna J J and AZ COVID ...
3,RT @HardeepSPuri: 🔬 Government further scales ...,RT Government further scales up India s Vaccin...
4,RT @HardeepSPuri: 🔬 Government further scales ...,RT Government further scales up India s Vaccin...


In [17]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x: analyze_sentiment(x))

In [18]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT Government further scales up India s Vaccin...,Neutral
1,RT Government further scales up India s Vaccin...,Neutral
2,RT Great News Pfizer Moderna J J and AZ COVID ...,Positive
3,RT Government further scales up India s Vaccin...,Neutral
4,RT Government further scales up India s Vaccin...,Neutral


In [19]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [20]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT Government further scales up India s Vaccin...,Neutral
1,RT Government further scales up India s Vaccin...,Neutral
2,RT Great News Pfizer Moderna J J and AZ COVID ...,Positive
3,RT Government further scales up India s Vaccin...,Neutral
4,RT Government further scales up India s Vaccin...,Neutral


In [21]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @HardeepSPuri: 🔬 Government further scales ...,Beinginformed.in,18915,293,My Country India,False,0,44,2021-07-04 17:46:31,RT Government further scales up India s Vaccin...,Neutral
1,RT @HardeepSPuri: 🔬 Government further scales ...,Mohsin Shaikh🇮🇳,13722,1805,"Pune, Maharashtra",False,0,44,2021-07-04 17:44:42,RT Government further scales up India s Vaccin...,Neutral
2,"RT @FaheemYounus: Great News:\n\nPfizer, Moder...",Ibn Anjar™,35074,874,"ÜT: -6.2130817,107.0288138",False,0,1539,2021-07-04 17:44:26,RT Great News Pfizer Moderna J J and AZ COVID ...,Positive
3,RT @HardeepSPuri: 🔬 Government further scales ...,VISHWANATH ACHARYA,26857,546,"Pune, Maharashtra, India",False,0,44,2021-07-04 17:44:19,RT Government further scales up India s Vaccin...,Neutral
4,RT @HardeepSPuri: 🔬 Government further scales ...,Astro-कुमकुम 🇮🇳,41656,1218,"New Delhi, India",False,0,44,2021-07-04 17:44:07,RT Government further scales up India s Vaccin...,Neutral


In [22]:
n = 20
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
Indian hospital closed amid FAKE COVID VACCINE scam, as 2,500+ are feared to have been given saline &amp; antibiotics i… https://t.co/vMJlF5TaIM

Clean Tweet:
Indian hospital closed amid FAKE COVID VACCINE scam as 2 500 are feared to have been given saline amp antibiotics i

Sentiment:
Negative


In [23]:
n = 202
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @NewsBytesApp: In a remarkable achievement, COVAXIN, Bharat Biotech's COVID-19 vaccine, is overall 77.8 percent effective, the company s…

Clean Tweet:
RT In a remarkable achievement COVAXIN Bharat Biotech s COVID 19 vaccine is overall 77 8 percent effective the company s

Sentiment:
Positive


In [24]:
n = 50
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @HardeepSPuri: 🔬 Government further scales up India’s Vaccine Development &amp; Manufacturing Ecosystem with support from PM-CARES.

🧪 Two m…

Clean Tweet:
RT Government further scales up India s Vaccine Development amp Manufacturing Ecosystem with support from PM CARES Two m

Sentiment:
Neutral


In [28]:
df[df.Sentiment=='Positive'].head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
2,"RT @FaheemYounus: Great News:\n\nPfizer, Moder...",Ibn Anjar™,35074,874,"ÜT: -6.2130817,107.0288138",False,0,1539,2021-07-04 17:44:26,RT Great News Pfizer Moderna J J and AZ COVID ...,Positive
7,"RT @FaheemYounus: Great News:\n\nPfizer, Moder...",AZ's 🇲🇾🌺,3998,20,,False,0,1539,2021-07-04 17:42:17,RT Great News Pfizer Moderna J J and AZ COVID ...,Positive
10,RT @PriyankKharge: India wasted valuable info ...,IRU PATIL ZALKI,12103,920,GULBARGA,False,0,19,2021-07-04 17:41:19,RT India wasted valuable info on 300 million C...,Positive
14,RT @c400_t: India: post vaccination death #262...,Droh Shankar,7778,202,,False,0,7,2021-07-04 17:40:02,RT t India post vaccination death 2626 Bihar M...,Positive
17,RT @AmitShah: Incredible!\n\nAnother feather i...,भूपेन्द्र पान्डेय,3509,493,प्रयागराज,False,0,5546,2021-07-04 17:38:00,RT Incredible Another feather in the cap of In...,Positive


In [25]:
df[df.Sentiment=='Positive'].shape[0]

362

In [29]:
df[df.Sentiment=='Neutral'].head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @HardeepSPuri: 🔬 Government further scales ...,Beinginformed.in,18915,293,My Country India,False,0,44,2021-07-04 17:46:31,RT Government further scales up India s Vaccin...,Neutral
1,RT @HardeepSPuri: 🔬 Government further scales ...,Mohsin Shaikh🇮🇳,13722,1805,"Pune, Maharashtra",False,0,44,2021-07-04 17:44:42,RT Government further scales up India s Vaccin...,Neutral
3,RT @HardeepSPuri: 🔬 Government further scales ...,VISHWANATH ACHARYA,26857,546,"Pune, Maharashtra, India",False,0,44,2021-07-04 17:44:19,RT Government further scales up India s Vaccin...,Neutral
4,RT @HardeepSPuri: 🔬 Government further scales ...,Astro-कुमकुम 🇮🇳,41656,1218,"New Delhi, India",False,0,44,2021-07-04 17:44:07,RT Government further scales up India s Vaccin...,Neutral
5,RT @amarprasadreddy: India administered over 3...,Santosh Dixit,20,0,,False,0,77,2021-07-04 17:44:03,RT India administered over 350 million COVID 1...,Neutral


In [26]:
df[df.Sentiment=='Neutral'].shape[0]

495

In [30]:
df[df.Sentiment=='Negative'].head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
12,@Prayagwale Why did the US refused to provide ...,Qurat ul Ain Ismail,4926,378,,False,0,0,2021-07-04 17:40:47,Why did the US refused to provide raw material...,Negative
20,Indian hospital closed amid FAKE COVID VACCINE...,Philip John Edwards,6859,357,Cambridge UK,False,0,0,2021-07-04 17:35:36,Indian hospital closed amid FAKE COVID VACCINE...,Negative
22,"RT @Indian10000000: ++ While affidavit says, 1...",NITYA NAND MISHRA,39436,127,"New Delhi, India",False,0,42,2021-07-04 17:34:20,RT While affidavit says 16cr doses on 5 May Cl...,Negative
44,"RT @Indian10000000: ++ While affidavit says, 1...",Parry Singh,14374,82,Heart,False,0,42,2021-07-04 17:22:12,RT While affidavit says 16cr doses on 5 May Cl...,Negative
66,Indian hospital closed amid FAKE COVID VACCINE...,Captain_N,37789,1302,Pacific,False,0,0,2021-07-04 17:14:19,Indian hospital closed amid FAKE COVID VACCINE...,Negative


In [27]:
df[df.Sentiment=='Negative'].shape[0]

143